<a href="https://colab.research.google.com/github/filipefbn/jupyter_intro_class_ibd/blob/master/notebooks/Exerc%C3%ADcios%20parte%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import sqlite3
import pandas as pd

## Exercícios

Neste exercício, vamos trabalhar com dados de uma empresa fictícia de consultoria denominada **IBD**.

**Considere o seguinte esquema relacional:**

![alt text](https://raw.githubusercontent.com/filipefbn/jupyter_intro_class_ibd/master/data/schemaexercicio.png)

Nenhum atributo admite valor nulo.

Apenas as seguintes chaves primárias são numéricas: `Cliente.CodCli`, `Consultor.CPF`


### Download dos dados da empresa

In [2]:
!wget --quiet -P /tmp/ https://raw.githubusercontent.com/filipefbn/jupyter_intro_class_ibd/master/data/exercicio.sql

### Carregando os dados

In [3]:
conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

In [4]:
f = io.open('/tmp/exercicio.sql', 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

In [40]:
cliente = pd.read_sql_query("SELECT * FROM Cliente", conn)
projeto = pd.read_sql_query("SELECT * FROM Projeto", conn)
consultor = pd.read_sql_query("SELECT * FROM Consultor", conn)
equipe = pd.read_sql_query("SELECT * FROM Equipe", conn)
viagem = pd.read_sql_query("SELECT * FROM Viagem", conn)
local = pd.read_sql_query("SELECT * FROM Local", conn)

## Consultas

### 1. Que consultores (CPF, nome e estado de nascimento) não nasceram em São Paulo (SP)?


In [5]:
df = pd.read_sql_query("SELECT C.CPF, C.NomeCon, C.DataNascCon"
                       + " FROM Consultor AS C"
                       + " WHERE C.UFNASCCON <> 'SP'", conn)
df

,CPF,NOMECON,DATANASCCON
0,13414263351,Samuel Araujo Azevedo,1990-09-25 11:40:27
1,71960563530,Diogo Cavalcanti Rocha,1991-09-25 11:40:27
2,3074973994,Julieta Maciel Sousa,1984-09-25 11:40:27
3,29762035631,Caio Dias Ribeiro,1997-09-25 11:40:27
4,17787063122,Larissa Fernandes Araujo,1977-09-25 11:40:28
5,31266012516,Leonor Goncalves Rocha,1979-09-25 11:40:28
6,85726311353,Brenda Oliveira Goncalves,1981-09-25 11:40:28


### 2. Que projetos (sigla e nome) não são do cliente "Facebook"?


In [6]:
df = pd.read_sql_query("SELECT P.Sigla, P.NomePro"
                       + " FROM Projeto AS P"
                       + " WHERE P.CodCli <> (SELECT C.CodCli"
                                             + " FROM Cliente AS C"
                                             + " WHERE C.NomeCli = 'Facebook')"
                       , conn)
df

,SIGLA,NOMEPRO
0,NMT,Neural Machine Translation (Chinês para Portug...
1,Search,Otimização Sistema de Busca
2,Spam,Detecção de Spam (Gmail)
3,Fake,Detecção de Fake News
4,SDC,Self Driving Car
5,Pay,Sistema de Pagamento
6,RecSys,Sistema de Recomendação
7,Fraud,Detecção de Fraude


### 3. Para que UFs viajaram os consultores que participam do projeto de sigla "RecSys"?

In [37]:
df = pd.read_sql_query("SELECT UFLoc"
                      + " FROM Local L"
                      + " INNER JOIN Viagem V"
                      + " ON L.NomeLoc = V.Destino"
                      + " WHERE V.Sigla = 'RecSys'"
                      + " GROUP BY UFLoc"
                      , conn)
df

,UFLOC
0,MG
1,SP


### 4. Que projetos (sigla e nome) não têm equipe?

In [48]:
df = pd.read_sql_query("SELECT P.Sigla, P.NomePro"
                       + " FROM Projeto P"
                       + " LEFT JOIN Equipe E"
                       + " ON P.Sigla = E.Sigla"
                       + " WHERE E.Sigla is null"
                       , conn)
df

,SIGLA,NOMEPRO
0,Search,Otimização Sistema de Busca
1,Fake,Detecção de Fake News
2,SDC,Self Driving Car


### 5. Que projetos (sigla e nome) não têm em sua equipe consultores nascidos antes de 1987?

In [72]:
df = pd.read_sql_query("SELECT *"
                       + " FROM Projeto P"
                       + " WHERE P.Sigla <> (SELECT *"
                                             + " FROM Equipe E"
                                             + " INNER JOIN (SELECT *"
                                                             + " FROM Consultor"
                                                             + " WHERE DataNascCon < '1987-01-01 00:00:00') as C"
                                             + " ON E.CPF = C.CPF"
                                             + " GROUP BY E.Sigla)"
                       , conn)
df

DatabaseError: Execution failed on sql 'SELECT * FROM Projeto P WHERE P.Sigla <> (SELECT * FROM Equipe E INNER JOIN (SELECT * FROM Consultor WHERE DataNascCon < '1987-01-01 00:00:00') as C ON E.CPF = C.CPF GROUP BY E.Sigla)': row value misused

In [50]:
consultor

,CPF,NOMECON,DATANASCCON,UFNASCCON
0,13414263351,Samuel Araujo Azevedo,1990-09-25 11:40:27,PR
1,71960563530,Diogo Cavalcanti Rocha,1991-09-25 11:40:27,PB
2,44434353608,Melissa Carvalho Cunha,1994-09-25 11:40:27,SP
3,62465487841,Lucas Fernandes Costa,1998-09-25 11:40:27,SP
4,3074973994,Julieta Maciel Sousa,1984-09-25 11:40:27,RJ
5,29762035631,Caio Dias Ribeiro,1997-09-25 11:40:27,MG
6,34646193892,Rodrigo Araujo Cardoso,2000-09-25 11:40:28,SP
7,17787063122,Larissa Fernandes Araujo,1977-09-25 11:40:28,MG
8,31266012516,Leonor Goncalves Rocha,1979-09-25 11:40:28,MG
9,85726311353,Brenda Oliveira Goncalves,1981-09-25 11:40:28,TO


In [49]:
equipe

,SIGLA,CPF
0,NMT,44434353608
1,Fraud,44434353608
2,Fraud,71960563530
3,Spam,44434353608
4,Spam,71960563530
5,RecSys,29762035631
6,RecSys,3074973994
7,RecSys,62465487841
8,WhatsApp,17787063122
9,WhatsApp,31266012516


In [64]:
projeto

,SIGLA,NOMEPRO,DATAINICIO,CODCLI
0,NMT,Neural Machine Translation (Chinês para Portug...,2019-09-24 11:40:26,1
1,Search,Otimização Sistema de Busca,2019-09-24 21:00:00,2
2,Spam,Detecção de Spam (Gmail),2016-09-25 11:40:26,2
3,Fake,Detecção de Fake News,2019-09-24 21:00:00,3
4,SDC,Self Driving Car,2019-09-24 21:00:00,3
5,Pay,Sistema de Pagamento,2019-08-22 11:40:27,6
6,RecSys,Sistema de Recomendação,2019-08-17 11:40:27,6
7,WhatsApp,Cliente desktop para o WhatsApp,2019-08-14 11:40:27,4
8,VR,Hardware para Oculus VR,2019-06-25 11:40:27,4
9,Fraud,Detecção de Fraude,2017-09-25 11:40:27,5


### 6. Que consultores (CPF e nome) viajaram por seu estado natal (origem ou destino)?


### 7. Para que locais não houve nenhuma viagem?

### 8. Em que projetos (sigla e nome) foram realizadas viagens para a California (CA) desde o início de 2019?
OBS: Como o banco foi populado usando a função [`datetime`](https://www.sqlite.org/lang_datefunc.html) do SQLite para as datas, podemos usar DataViagem > '2019' com o `WHERE` (ver link para mais informações).

### 9. Que consultores (CPF e nome) viajaram por um projeto de cuja equipe não fazem parte?
OBS: Apesar de não intuitivo, isso é possível, pois no nosso schema temos que Viagem.CPF 🡪 Consultor.CPF e Viagem.Sigla 🡪 Projeto.Sigla.

### 10. Que consultores (CPF e nome) não estão em nenhuma equipe de projeto que tenha iniciado em 2019?
